In [5]:
import pandas as pd
import numpy as np
import ast
import redis
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware

# connect Redis databases
WATCHDOG_ADDRESS = "192.168.226.176"
db1 = redis.StrictRedis(
    host=WATCHDOG_ADDRESS,
    port=6379,
    db=1)
db1_keys = db1.keys()

db2 = redis.StrictRedis(
    host=WATCHDOG_ADDRESS,
    port=6379,
    db=2)
db2_keys = db2.keys()
genesis = ast.literal_eval(db2.get(b'genesis').decode('utf-8'))

# construct dataframe
rows = []
for key in db1_keys:
    row = []
    # IP address of node
    ip = key.decode("utf-8")
    # check if node is validator
    value = ast.literal_eval(db1.get(key).decode('utf-8'))
    is_validator = value['key'][2:] in genesis['extraData']
    # find the number of peers
    url = 'http://'+ip+':8545'
    provider = Web3.HTTPProvider(url)
    w3 = Web3(provider)
    w3.middleware_onion.inject(geth_poa_middleware, layer=0)
    peers_num = len(w3.geth.admin.peers())

    row.append(value['hostname'])
    row.append(ip)
    row.append(value['key'])
    row.append(is_validator)
    row.append(peers_num)
    
    rows.append(row)
df_orig = pd.DataFrame(np.array(rows),
    columns=['Hostname', 'IP', 'NodeAddress', 'IsValidator', 'PeersNum'])

# sort dataframe
df = df_orig.copy()
idx = []
for name in df.Hostname:
    idx.append(int(name.split('-')[1]))
df['Index'] = idx
df = df.set_index(keys=df.Index).drop(labels='Index', axis=1).sort_index()
df

,Hostname,IP,NodeAddress,IsValidator,PeersNum
Index,,,,,
1,besu-1,192.168.226.173,0x8b5a4561ccb044e345b20b56769a943444df490d,True,8
2,besu-2,192.168.226.86,0x1037869b64dabba6ef9ad5ac3f679ebc136d1b6c,True,7
3,besu-3,192.168.226.124,0xe7db4efc5c65f82287d045613557a8d5ed94dca9,True,7
4,besu-4,192.168.226.233,0xa422a3003a0a26c548be93d8de5804eaa3dd644f,True,9
5,besu-5,192.168.226.89,0x9a42b2652316cce01efaff91171430794e9ffc5e,True,9
6,besu-6,192.168.226.128,0x79753210c2c726d2eee7b79146ac8e5e3ff06687,True,11
7,besu-7,192.168.226.69,0x8b08afd8f8207de9e11c8bdaca083162ffbb0968,True,10
8,besu-8,192.168.226.52,0xe7e8b25500086ca761d9c0681144d409a0d3244a,True,7


In [16]:
import yaml
host_name = 'node-4'
host_ip = '102.168.226.75'
with open('../deploy/monitor/prometheus-template.yml') as f:
    prometheus = yaml.safe_load(f)
prometheus

{'global': {'scrape_interval': '15s'},
 'scrape_configs': [{'job_name': 'prometheus',
   'static_configs': [{'targets': ['localhost:9090']}]},
  {'job_name': 'push-gateway',
   'metrics_path': '/metrics',
   'honor_labels': True,
   'scheme': 'http',
   'static_configs': [{'targets': ['localhost:9091']}]}]}

In [20]:
name = prometheus['scrape_configs'][1]['job_name']
prometheus['scrape_configs'][1]['job_name'] = name + '-' + host_name
prometheus['scrape_configs'][1]['static_configs'][0]['targets'][0] = host_ip + ':9091'
prometheus

{'global': {'scrape_interval': '15s'},
 'scrape_configs': [{'job_name': 'prometheus',
   'static_configs': [{'targets': ['localhost:9090']}]},
  {'job_name': 'push-gateway-node-4-node-4',
   'metrics_path': '/metrics',
   'honor_labels': True,
   'scheme': 'http',
   'static_configs': [{'targets': ['102.168.226.75:9091']}]}]}